In [2]:
import pandas as pd
import geopandas as gpd
import rasterio
from cartopy import crs
from cartopy.io.img_tiles import MapboxTiles
from shapely.geometry import Point
from folium import GeoJson
from folium import Map
from numpy import mean
import geojson
import json

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (40,40)
token = 'pk.eyJ1IjoibWFwYm94IiwiYSI6ImNpejY4M29iazA2Z2gycXA4N2pmbDZmangifQ.-g_vE53SD2WrJ6tFX7QHmA'
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


# Mt. Rainier Snow Cover Locations

from JHRL/Ian

In [3]:
snowcover = pd.read_csv("../data/snow_summary_all_2009_2017_locs.csv")
snowcover['geometry'] = [Point(xy) for xy in zip(snowcover.longitude, snowcover.latitude)]
snowcover = gpd.GeoDataFrame(snowcover)
snowcover.crs = {"init" : 'epsg:4326'}
bounds = snowcover.cascaded_union.buffer(0.01).bounds
bounds = [
    bounds[0],
    bounds[2],
    bounds[1],
    bounds[3]
]


In [ ]:
tiler = MapboxTiles(token, 'mapbox.streets')
mercator = tiler.crs
ax = plt.axes(projection=mercator)
ax.set_extent(bounds)
ax.add_geometries(snowcover.dropna().geometry.buffer(0.001), crs=crs.PlateCarree(), color='red')
ax.add_image(tiler, 13)



In [3]:
json = GeoJson(snowcover.dropna().to_json())
center = [mean(snowcover.latitude), mean(snowcover.longitude)]
m = Map(center)
json.add_to(m)
m.save('locations.html')

In [20]:
!du -h *

1.2M	locations.html
524K	Untitled.ipynb


In [4]:
unique = snowcover.drop_duplicates('Location').dropna()
small_json = GeoJson(unique.to_json())
center = [mean(unique.latitude), mean(unique.longitude)]
m = Map(center)
small_json.add_to(m)


In [5]:
gpd.casc

AttributeError: module 'geopandas' has no attribute 'casc'

In [5]:
coveredarea = unique.cascaded_union.convex_hull

In [6]:
geojson.Feature(geometry=coveredarea).geometry.coordinates

[[[-121.91632, 46.73899],
  [-121.91641000000001, 46.73915],
  [-121.91672, 46.739740000000005],
  [-121.88728, 46.994679999999995],
  [-121.53754920000002, 46.994992100000005],
  [-121.53741979999998, 46.99495506],
  [-121.5352794, 46.99410449],
  [-121.402, 46.888740000000006],
  [-121.55821, 46.741009999999996],
  [-121.91632, 46.73899]]]

In [7]:
!cat sensor_bbox.json

{"geometry": {"coordinates": [[[-121.91632, 46.73899], [-121.91641000000001, 46.73915], [-121.91672, 46.739740000000005], [-121.88728, 46.994679999999995], [-121.53754920000002, 46.994992100000005], [-121.53741979999998, 46.99495506], [-121.5352794, 46.99410449], [-121.402, 46.888740000000006], [-121.55821, 46.741009999999996], [-121.91632, 46.73899]]], "type": "Polygon"}, "properties": {}, "type": "Feature"}

In [8]:
geojson.Feature(geometry=coveredarea).geometry.coordinates

[[[-121.91632, 46.73899],
  [-121.91641000000001, 46.73915],
  [-121.91672, 46.739740000000005],
  [-121.88728, 46.994679999999995],
  [-121.53754920000002, 46.994992100000005],
  [-121.53741979999998, 46.99495506],
  [-121.5352794, 46.99410449],
  [-121.402, 46.888740000000006],
  [-121.55821, 46.741009999999996],
  [-121.91632, 46.73899]]]

In [13]:
from planet import api
aoi = {
    'type' : "Polygon",
    'coordinates' : geojson.Feature(geometry=coveredarea).geometry.coordinates

}
client = api.ClientV1()
query = api.filters.and_filter(
    api.filters.geom_filter(aoi), 
    api.filters.date_range('acquired', gt="2018-01-18"),
    api.filters.date_range('acquired', lt="2018-01-20")
)
# we are requesting PlanetScope 4 Band imagery
item_types = ['PSScene4Band']
request = api.filters.build_search_request(query, item_types)
# this will cause an exception if there are any API related errors
results = client.quick_search(request)

In [14]:
for item in results.items_iter(50):
    print(item['id'], item['properties']['cloud_cover'], item['properties']['published'], item['_links']['thumbnail'])

20180119_182528_0e3a 0.35 2018-01-20T10:28:21Z https://api.planet.com/data/v1/item-types/PSScene4Band/items/20180119_182528_0e3a/thumb
20180119_182527_0e3a 0.42 2018-01-20T08:11:33Z https://api.planet.com/data/v1/item-types/PSScene4Band/items/20180119_182527_0e3a/thumb
20180119_182526_0e3a 0.44 2018-01-20T08:11:33Z https://api.planet.com/data/v1/item-types/PSScene4Band/items/20180119_182526_0e3a/thumb
20180119_182525_0e3a 0.61 2018-01-20T08:11:33Z https://api.planet.com/data/v1/item-types/PSScene4Band/items/20180119_182525_0e3a/thumb
20180119_182503_0f52 0.24 2018-01-20T04:03:41Z https://api.planet.com/data/v1/item-types/PSScene4Band/items/20180119_182503_0f52/thumb
20180119_182502_0f52 0.28 2018-01-20T04:03:41Z https://api.planet.com/data/v1/item-types/PSScene4Band/items/20180119_182502_0f52/thumb
20180119_182500_0f52 0.37 2018-01-20T04:03:40Z https://api.planet.com/data/v1/item-types/PSScene4Band/items/20180119_182500_0f52/thumb
20180119_182501_0f52 0.32 2018-01-20T02:15:47Z https://